# Movies Exploratory Analysis
The purpose of this notebook is to explore the data ingested from the TMDB discover/movie endpoint. We will examine its characteristics (i.e. data types, number of missing values, etc) and plan the initial schema for the database.

In [1]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

# set the working directory to project root
load_dotenv()
PROJECT_ROOT = os.getenv("PROJECT_ROOT")
os.chdir(PROJECT_ROOT)

# set data_path
DATA_PATH = "data/movies.csv"

In [2]:
print(os.getcwd())

/home/jjacob/movie_data_analysis


In [4]:
# read data into pandas
df = pd.read_csv(DATA_PATH, engine='python')

In [6]:
# print summary statistics and a 5 row sample of the data
df.info
df.describe()
print(df.head())

   adult                     backdrop_path                genre_ids     id  \
0  False  /7isarjYDEKZ5t1CgcvbuqEUby8P.jpg                     [27]   9532   
1  False  /Ar7QuJ7sJEiC0oP3I8fKBKIQD9u.jpg             [28, 18, 12]     98   
2  False  /mZj8EUr6F1x2PWZjKPxaeYd5WRw.jpg  [12, 16, 35, 10751, 14]  11688   
3  False  /24DZfupDlhXeTchmcOkoGRhP5Vg.jpg             [12, 28, 53]    955   
4  False   /zvmsyAMr3cVDdIu7UvDLSmRXlF.jpg          [35, 18, 10749]  22705   

  original_language            original_title  \
0                en         Final Destination   
1                en                 Gladiator   
2                en  The Emperor's New Groove   
3                en    Mission: Impossible II   
4                it             Tra(sgre)dire   

                                            overview  popularity  \
0  After a teenager has a terrifying vision of hi...     25.9950   
1  After the death of Emperor Marcus Aurelius, hi...     17.7690   
2  When self-centered Emperor Ku